In [1]:
#!/usr/bin/env python
# coding: utf-8

from pathlib import Path
import pandas as pd
import numpy as np
import sys
import os

In [2]:
Path('SALIDAS1').mkdir(parents=True, exist_ok=True)
lista_files = os.listdir('DATA_ORIGINAL/')
def imputar(archivo_csv):
    print(''); print('Procesando', archivo_csv)
    data            = pd.read_csv('DATA_ORIGINAL/' + archivo_csv, index_col = 0, parse_dates=True)
    data_nan        = data.fillna(999)                                         
    promedios_horas = data.groupby(data.index.strftime('%H')).mean()

    suma = 0
    for name_estacion in data.columns:
        suma += 1; progress = int(suma/len(data.columns) * 100)
        sys.stdout.write('\r')
        sys.stdout.write('[%-20s] %d%%' % ('=' * progress, progress))
        sys.stdout.flush()

        for i in np.arange(0,len(data_nan[name_estacion]),1):
            dato = data_nan[name_estacion][i]

            if dato == 999:
                hora = int(data_nan[name_estacion].index[i].hour)
                data_nan[name_estacion][i] = promedios_horas[name_estacion][hora]
    # QUITAR esta linea para tener todas las estaciones
   # data_nan = pd.DataFrame(data_nan.mean(axis=1)).rename(columns={0: 'MEAN'})
    data_nan.to_csv('SALIDAS1/llenado_' + archivo_csv)


In [3]:
lista_files = ['CO.csv','DIR.csv','HR.csv','NO.csv','O3.csv','TEM.csv','VEL.csv','NO2.csv','PM10.csv','PM25.csv','RAD.csv']
for i in lista_files:
    imputar(i)


Procesando CO.csv
[====================================================================================================] 100%
Procesando DIR.csv
[====================================================================================================] 100%
Procesando HR.csv
[====================================================================================================] 100%
Procesando NO.csv
[====================================================================================================] 100%
Procesando O3.csv
[====================================================================================================] 100%
Procesando TEM.csv
[====================================================================================================] 100%
Procesando VEL.csv
[====================================================================================================] 100%
Procesando NO2.csv


C:\Users\Diana G\AppData\Local\Temp\ipykernel_11248\691693840.py:5: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  data            = pd.read_csv('DATA_ORIGINAL/' + archivo_csv, index_col = 0, parse_dates=True)


[=====               ] 5%

C:\Users\Diana G\AppData\Local\Temp\ipykernel_11248\691693840.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_nan[name_estacion][i] = promedios_horas[name_estacion][hora]


[======================================================================] 70%

KeyError: 'SUBA'

In [4]:
lista_files = ['PM10.csv','PM25.csv','RAD.csv']
for i in lista_files:
    imputar(i)


Procesando PM10.csv
[====================================================================================================] 100%
Procesando PM25.csv
[====================================================================================================] 100%
Procesando RAD.csv
[====================================================================================================] 100%

In [5]:
lista_files = ['NO2.csv']
for i in lista_files:
    imputar(i)


Procesando NO2.csv
[====================================================================================================] 100%

In [6]:
print(d.strftime('%m,/%d/%Y'))

NameError: name 'd' is not defined

In [2]:
def mean_axis1(archivo_csv):
    data = pd.read_csv('DATA_ORIGINAL/' + archivo_csv, index_col = 0, parse_dates=True)
    mean = data.mean(axis=1)
    mean = np.array(mean)
    inds = np.where(np.isnan(mean))
    mean[inds] = np.nanmean(mean)
    mean = pd.DataFrame({'MEAN':mean})
    mean.to_csv('SALIDAS1/llenado_' + archivo_csv)
    
def components(ws,wd):
    
    ut = np.ones((len(wd),len(wd.columns)))
    vt = np.ones((len(wd),len(wd.columns)))
    
    for i,col in enumerate(wd.columns):
        #Convert to components!
        u_wind = np.array(ws['MEAN']) * np.sin(np.array(wd[col]) * np.pi/180)
        v_wind = np.array(ws['MEAN']) * np.cos(np.array(wd[col]) * np.pi/180)
        #Add the components to a matrix
        ut[:,i] = u_wind
        vt[:,i] = v_wind
      
    #Calculate the stations mean and replace the nan values!
    u_wind = np.nanmean(ut,axis=1)
    inds = np.where(np.isnan(u_wind))
    u_wind[inds] = np.nanmean(u_wind)
    v_wind = np.nanmean(vt,axis=1)
    inds = np.where(np.isnan(v_wind))
    v_wind[inds] = np.nanmean(v_wind)
    
    ## Add U
    df0 = pd.DataFrame(np.column_stack((ws['FECHA'],u_wind)), columns=['FECHA','MEAN'])
    df0.to_csv('SALIDAS1/llenado_u-component.csv',index=False)
    
    ## Add V
    df1 = pd.DataFrame(np.column_stack((ws['FECHA'],v_wind)), columns=['FECHA','MEAN'])
    df1.to_csv('SALIDAS1/llenado_v-component.csv',index=False)
    
    #Calculate the wind direction using the wind components!
    mean_wd = np.arctan2(u_wind, v_wind) * 180/np.pi
    #Result is from -180 to 180 due to arctan2, this move it between 0 to 360
    mean_wd = (180 + mean_wd)
    df2 = pd.DataFrame(np.column_stack((ws['FECHA'],mean_wd)), columns=['FECHA','MEAN'])
    df2.to_csv('SALIDAS1/llenado_DIR.csv',index=False)
    return(mean_wd)

def change_dates(filee):
    df0 = pd.read_csv('DATA_ORIGINAL/NO.csv')
    df1 = pd.read_csv('DATA_ORIGINAL/'+filee)
    df1['FECHA'] = df0['FECHA']
    df1.to_csv('DATA_ORIGINAL/'+filee, index = False)

In [3]:
Path('SALIDAS1').mkdir(parents=True, exist_ok=True)
lista_files = os.listdir('DATA_ORIGINAL/') 

chem = 'no'
if chem == 'yes':
    lista_files = ['VEL.csv']
    for i in lista_files:
        change_dates(i)
        imputar(i)

In [4]:
chem = 'no'
if chem == 'yes':
    lista_files = ['TEM.csv']
    for i in lista_files:
        mean_axis1(i)

In [5]:
data_ws = pd.read_csv('SALIDAS1/llenado_VEL.csv')
change_dates('DIR.csv')
data_dir = pd.read_csv('DATA_ORIGINAL/DIR.csv',index_col=0,parse_dates=True)

wind = components(data_ws,data_dir)

KeyError: 'MEAN'

In [6]:
wind

NameError: name 'wind' is not defined